In [1]:
import torch
path_to_pth_file = "/content/mnist_cnn_model_simba.pth"
model = torch.load(path_to_pth_file)

import keras.datasets.mnist as mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

FileNotFoundError: [Errno 2] No such file or directory: '/content/mnist_cnn_model_simba.pth'

In [29]:
import numpy as np

conv1_wt=model['conv1.weight']
conv1_bias=model['conv1.bias']
fc1_wt=model['fc1.weight']
fc1_bias=model['fc1.bias']
fc2_wt=model['fc2.weight']
fc2_bias=model['fc2.bias']

conv1_wt = conv1_wt/255

conv1_wt=conv1_wt.numpy()
conv1_bias=conv1_bias.numpy()
fc1_wt=fc1_wt.numpy()
fc1_bias=fc1_bias.numpy()
fc2_wt=fc2_wt.numpy()
fc2_bias=fc2_bias.numpy()

In [46]:
def conv_2d(test_images, conv_wt, conv_bias):
    conv_outputs = []
    imgs = np.expand_dims(test_images, axis=1)
    batch_size, in_channels, in_height, in_width = imgs.shape
    out_channels, _, kernel_height, kernel_width = conv_wt.shape
    out_height = in_height - kernel_height + 1
    out_width = in_width - kernel_width + 1
    for img in imgs:
        img_conv_outputs = []
        for filter, b in zip(conv_wt, conv_bias):
            conv_output = np.zeros((out_height, out_width))
            for i in range(out_height):
                for j in range(out_width):
                    roi = img[:, i:i+kernel_height, j:j+kernel_width]
                    conv_output[i, j] = np.sum(roi * filter) + b
            img_conv_outputs.append(conv_output)

        conv_outputs.append(img_conv_outputs)

    conv_outputs = np.array(conv_outputs)
    return conv_outputs

In [31]:
def relu(input_data):
    return np.maximum(input_data, 0)

def fully_connected(input_data, weights, bias):
    return np.matmul(input_data, weights.T) + bias

def softmax(input_data):
    exp_values = np.exp(input_data - np.max(input_data, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

In [48]:
def max_pool(conv_out, kernel_size):
    batch_size, channels, height, width = conv_out.shape
    out_height = height // kernel_size
    out_width = width // kernel_size

    output = np.zeros((batch_size, channels, out_height, out_width))

    for b in range(batch_size):
        for c in range(channels):
            for i in range(out_height):
                for j in range(out_width):
                    max_val = np.max(conv_out[b, c, kernel_size*i: kernel_size*(i+1), kernel_size*j: kernel_size*(j+1)])
                    output[b, c, i, j] = max_val
    return output

In [42]:

def forward_pass(input_data, conv1_wt, conv1_bias, fc1_wt, fc1_bias, fc2_wt, fc2_bias):

    print("input: ", input_data.shape)

    conv_output = conv_2d(input_data, conv1_wt, conv1_bias)
    print("convolution: ", conv_output.shape)

    conv_output = relu(conv_output)
    print("activation: ", conv_output.shape)

    pooled_output = max_pool(conv_output, kernel_size=2)
    print("max pooling: ", pooled_output.shape)

    flattened_output = pooled_output.reshape(len(input_data), -1)
    print("flattened_output: ", flattened_output.shape)


    fc1_output = fully_connected(flattened_output, fc1_wt, fc1_bias)
    print("fully connected layer 1:", fc1_output.shape)

    fc1_output = relu(fc1_output)

    fc2_output = fully_connected(fc1_output, fc2_wt, fc2_bias)
    print("fully connected layer 2:", fc2_output.shape)

    probabilities = softmax(fc2_output)

    predicted_classes = np.argmax(probabilities, axis=1)

    return predicted_classes

In [53]:
pred = forward_pass(test_images[:100], conv1_wt, conv1_bias, fc1_wt, fc1_bias, fc2_wt, fc2_bias)

print("Predicted values: ", pred[:10])
print("Test Labels     : ", test_labels[:10])
print("accuracy: ", sum(pred[:100] == test_labels[:100])/len(pred[:100]))

input:  (100, 28, 28)
convolution  (100, 32, 26, 26)
activation:  (100, 32, 26, 26)
max pooling:  (100, 32, 13, 13)
flattened_output:  (100, 5408)
fully connected layer 1: (100, 64)
fully connected layer 2: (100, 10)
Predicted values:  [7 2 1 0 4 1 4 9 6 9]
Test Labels     :  [7 2 1 0 4 1 4 9 5 9]
accuracy:  0.86
